In [1]:
#Include the neccessary librarys
import numpy as np #adds support for matrices
import random #adds support for the any of the ai's random moves (ie. the first turn)
import pygame #adds support for game's visual interface
import pygame_menu #adds support for menu and button functions
from pygame_menu.examples import create_example_window # the function used for this example saves time and provides clarity in the code
from typing import Tuple, Any, Optional, List
from random import randrange #for game background color
import sys #adds support for 
import math #helps determine which column a peice should be dropped based on mouse position


pygame 2.6.0 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame-menu 4.4.3


In [2]:
#Each RGB (on the right) gives a corresponding color value to each of the color name variables
BLUE = (0,15,255)
BLACK = (21,21,21)
RED = (234,32,0)
YELLOW = (255,236,0)

In [3]:
ROW_COUNT = 6 #Determines number of horizontal slots
COL_COUNT = 7 #Determines number of vertical slots

In [4]:
# Assigns a constant which determines turn to each player
PLAYER = 0 
AI = 1

# Assigns a constant to determine if a slot is filled and by who
EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2

In [5]:
WINDOW_LENGTH = 4 #sets number of consecutive pieces needed to win to 4 by default
WINDOW_SIZE = (640, 480) #pygame screen size
GAME_MODE = ['CLASSIC'] #sets game mode to classic by default
game_mode = ['CLASSIC', 'CONNECT5']

In [6]:
#generates a matrix to represent the board slot setup
def create_board():
    board = np.zeros ((ROW_COUNT, COL_COUNT))
    return board

#generates a random color (will be used later for background)
def random_color() -> Tuple[int, int, int]:
    return randrange(0, 255), randrange(0, 255), randrange(0, 255)

#determines which location to place a piece 
def drop_piece(board, row, col, piece):
    piece = board[row][col]

#checks if a given column is full 
def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

#checks for the bottom-most available row in a given column
def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
    	if board[r][col] == 0:
    		return r

#displays most recent state of the game board
def print_board(board):
    print(np.flip(board, 0))

In [7]:

# Updates game mode to match user selection
def change_game_mode(value: Tuple[Any, int], game_mode: str) -> None:
    selected, index = value
    print(f'Selected gamemode: "{selected}" ({game_mode}) at index {index}')
    GAME_MODE[0] = game_mode

# Specifies the functions of each game mode when selected
def play_function(game_mode: List[str], font: 'pygame.font.Font', test: bool = False) -> None:
    assert isinstance(game_mode, list) and game_mode, "Game mode must be a non-empty list."
    game_mode = game_mode[0]
    assert isinstance(game_mode, str), "Game mode must be a string."

    global main_menu, clock

    ROW_COUNT, COL_COUNT, WINDOW_LENGTH = (6, 7, 4) if game_mode == 'CLASSIC' else (8, 9, 5)

    def winning_move(board, piece):
        # Check horizontal, vertical, and diagonal locations for win
        for c in range(COL_COUNT - WINDOW_LENGTH + 1):
            for r in range(ROW_COUNT):
                if all(board[r][c + i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COL_COUNT):
            for r in range(ROW_COUNT - WINDOW_LENGTH + 1):
                if all(board[r + i][c] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COL_COUNT - WINDOW_LENGTH + 1):
            for r in range(ROW_COUNT - WINDOW_LENGTH + 1):
                if all(board[r + i][c + i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COL_COUNT - WINDOW_LENGTH + 1):
            for r in range(WINDOW_LENGTH - 1, ROW_COUNT):
                if all(board[r - i][c + i] == piece for i in range(WINDOW_LENGTH)):
                    return True

    def evaluate_window(window, piece):
        score = 0
        opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

        if window.count(piece) == WINDOW_LENGTH:
            score += 100
        elif window.count(piece) == WINDOW_LENGTH - 1 and window.count(EMPTY) == 1:
            score += 5
        elif window.count(piece) == WINDOW_LENGTH - 2 and window.count(EMPTY) == 2:
            score += 2

        if window.count(opp_piece) == WINDOW_LENGTH - 1 and window.count(EMPTY) == 1:
            score -= 4

        return score

    if game_mode not in ['CLASSIC', 'CONNECT5']:
        raise ValueError(f'Unknown game_mode: {game_mode}')

    f_esc = font.render('Press ESC to open the menu', True, (255, 255, 255))
    bg_color = random_color()

    main_menu.disable()
    main_menu.full_reset()

    frame = 0

    while True:
        clock.tick(60)
        frame += 1

        events = pygame.event.get()
        for e in events:
            if e.type == pygame.QUIT:
                exit()
            elif e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE:
                main_menu.enable()
                return

        if main_menu.is_enabled():
            main_menu.update(events)

        surface.fill(bg_color)
        surface.blit(f, (int((WINDOW_SIZE[0] - f.get_width()) / 2), int(WINDOW_SIZE[1] / 2 - f.get_height())))
        surface.blit(f_esc, (int((WINDOW_SIZE[0] - f_esc.get_width()) / 2), int(WINDOW_SIZE[1] / 2 + f_esc.get_height())))
        pygame.display.flip()

        if test and frame == 2:
            break
    def is_terminal_node(board): #Checks if Player or AI has won or if there is a draw
        return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0
    def minimax(board, depth, alpha, beta, maximizingPlayer):
    	valid_locations = get_valid_locations(board)
    	is_terminal = is_terminal_node(board)
    	if depth == 0 or is_terminal:
    		if is_terminal:
    			if winning_move(board, AI_PIECE):
    				return (None, 100000000000000)
    			elif winning_move(board, PLAYER_PIECE):
    				return (None, -10000000000000)
    			else: # Game is over, no more valid moves
    				return (None, 0)
    		else: # Depth is zero
    			return (None, score_position(board, AI_PIECE))
    	if maximizingPlayer:
    		value = -math.inf
    		column = random.choice(valid_locations)
    		for col in valid_locations:
    			row = get_next_open_row(board, col)
    			b_copy = board.copy()
    			drop_piece(b_copy, row, col, AI_PIECE)
    			new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
    			if new_score > value:
    				value = new_score
    				column = col
    			alpha = max(alpha, value)
    			if alpha >= beta:
    				break
    		return column, value
    
    	else: # Minimizing player
    		value = math.inf
    		column = random.choice(valid_locations)
    		for col in valid_locations:
    			row = get_next_open_row(board, col)
    			b_copy = board.copy()
    			drop_piece(b_copy, row, col, PLAYER_PIECE)
    			new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
    			if new_score < value:
    				value = new_score
    				column = col
    			beta = min(beta, value)
    			if alpha >= beta:
    				break
    		return column, value
    SQUARESIZE = 100
    width = COL_COUNT * SQUARESIZE
    height = (ROW_COUNT + 1) * SQUARESIZE
    size = (width, height)
    RADIUS = int(SQUARESIZE / 2 - 5)
    
    screen = pygame.display.set_mode(size)
    draw_board(board)
    pygame.display.update()
    
    myfont = pygame.font.SysFont("8_bit", 75)
    turn = random.randint(PLAYER, AI)
    
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.MOUSEMOTION:
                pygame.draw.rect(screen, BLACK, (0, 0, width, SQUARESIZE))
                posx = event.pos[0]
                if turn == PLAYER:
                    pygame.draw.circle(screen, RED, (posx, int(SQUARESIZE / 2)), RADIUS)
            pygame.display.update()
    
            if event.type == pygame.MOUSEBUTTONDOWN:
                pygame.draw.rect(screen, BLACK, (0, 0, width, SQUARESIZE))
                if turn == PLAYER:
                    posx = event.pos[0]
                    col = int(math.floor(posx / SQUARESIZE))
    
                    if is_valid_location(board, col):
                        row = get_next_open_row(board, col)
                        drop_piece(board, row, col, PLAYER_PIECE)
    
                        if winning_move(board, PLAYER_PIECE):
                            label = myfont.render("You Win!!", 1, RED)
                            screen.blit(label, (40, 10))
                            game_over = True
    
                        turn += 1
                        turn = turn % 2
                        print_board(board)
                        draw_board(board)
    
        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)
    
            if is_valid_location(board, col):
                pygame.time.wait(500)
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI_PIECE)
    
                if winning_move(board, AI_PIECE):
                    label = myfont.render("Try Again!!", 1, YELLOW)
                    screen.blit(label, (40, 10))
                    game_over = True
    
                print_board(board)
                draw_board(board)
                turn += 1
                turn = turn % 2
    
        if game_over:
            pygame.time.wait(3000)

In [8]:
#Calculates score
def score_position(board, piece):
    score = 0
    
    ## Score center column
    center_array = [int(i) for i in list(board[:, COL_COUNT//2])]
    center_count = center_array.count(piece)
    score += center_count * 3
    
    ## Score Horizontal
    for r in range(ROW_COUNT):
    	row_array = [int(i) for i in list(board[r,:])]
    	for c in range(COL_COUNT-3):
    		window = row_array[c:c+WINDOW_LENGTH]
    		score += evaluate_window(window, piece)
    
    ## Score Vertical
    for c in range(COL_COUNT):
    	col_array = [int(i) for i in list(board[:,c])]
    	for r in range(ROW_COUNT-3):
    		window = col_array[r:r+WINDOW_LENGTH]
    		score += evaluate_window(window, piece)
    
    ## Score posiive sloped diagonal
    for r in range(ROW_COUNT-3):
    	for c in range(COL_COUNT-3):
    		window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
    		score += evaluate_window(window, piece)

    for r in range(ROW_COUNT-3):
    	for c in range(COL_COUNT-3):
    		window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
    		score += evaluate_window(window, piece)
    
    return score
    


def get_valid_locations(board):
	valid_locations = []
	for col in range(COL_COUNT):
		if is_valid_location(board, col):
			valid_locations.append(col)
	return valid_locations

def pick_best_move(board, piece):

	valid_locations = get_valid_locations(board)
	best_score = -10000
	best_col = random.choice(valid_locations)
	for col in valid_locations:
		row = get_next_open_row(board, col)
		temp_board = board.copy()
		drop_piece(temp_board, row, col, piece)
		score = score_position(temp_board, piece)
		if score > best_score:
			best_score = score
			best_col = col

	return best_col

def draw_board(board):
	for c in range(COL_COUNT):
		for r in range(ROW_COUNT):
			pygame.draw.rect(screen, BLUE, (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
			pygame.draw.circle(screen, BLACK, (int(c*SQUARESIZE+SQUARESIZE/2), int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)), RADIUS)
	
	for c in range(COL_COUNT):
		for r in range(ROW_COUNT):		
			if board[r][c] == PLAYER_PIECE:
				pygame.draw.circle(screen, RED, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
			elif board[r][c] == AI_PIECE: 
				pygame.draw.circle(screen, YELLOW, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
	pygame.display.update()

In [9]:
board = create_board()
print_board(board)
game_over = False

pygame.init()



[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


(5, 0)

In [10]:
# Constants
WINDOW_SIZE = (800, 600)  # Example window size
FPS = 60
ABOUT = ["This is a game.", "Version 1.0", "Developed by XYZ"]
GAME_MODE = 'CLASSIC'  # Default game mode

clock: Optional['pygame.time.Clock'] = None
main_menu: Optional['pygame_menu.Menu'] = None
surface: Optional['pygame.Surface'] = None
# Background color for main menu
def main_background() -> None:
    global surface
    surface.fill((128, 0, 128))

# Menu system program
def main_program(test: bool = False) -> None:
    global clock, main_menu, surface
    clock = pygame.time.Clock()
    surface = create_example_window('Connect4', WINDOW_SIZE)

    # Create menus
    play_menu = create_play_menu()
    main_menu = create_main_menu(play_menu)

    # Main loop
    while True:
        clock.tick(FPS)
        main_background()
        handle_events()

        if main_menu.is_enabled():
            main_menu.mainloop(surface, main_background, disable_loop=test, fps_limit=FPS)

        pygame.display.flip()
        if test:
            break

def create_play_menu() -> pygame_menu.Menu:
    play_menu = pygame_menu.Menu(
        height=int(WINDOW_SIZE[1] * 0.7),
        title='Play Menu',
        width=int(WINDOW_SIZE[0] * 0.75)
    )

    play_submenu = create_play_submenu()
    play_menu.add.button('Start', play_function, GAME_MODE, pygame.font.Font(pygame_menu.font.FONT_8BIT, 30))
    play_menu.add.selector('Select game mode ', [('1 - Classic', 'CLASSIC'), ('2 - Connect5', 'CONNECT5')],
                           onchange=change_game_mode, selector_id='select_game_mode')
    play_menu.add.button('Return to main menu', pygame_menu.events.BACK)

    return play_menu

def create_play_submenu() -> pygame_menu.Menu:
    submenu_theme = pygame_menu.themes.THEME_DEFAULT.copy()
    submenu_theme.widget_font_size = 15
    play_submenu = pygame_menu.Menu(
        height=int(WINDOW_SIZE[1] * 0.5),
        theme=submenu_theme,
        title='Submenu',
        width=int(WINDOW_SIZE[0] * 0.7)
    )
    for i in range(30):
        play_submenu.add.button(f'Back {i}', pygame_menu.events.BACK)
    play_submenu.add.button('Return to main menu', pygame_menu.events.RESET)
    return play_submenu

def create_main_menu(play_menu: pygame_menu.Menu) -> pygame_menu.Menu:
    main_theme = pygame_menu.themes.THEME_DEFAULT.copy()
    main_menu = pygame_menu.Menu(
        height=int(WINDOW_SIZE[1] * 0.6),
        theme=main_theme,
        title='Main Menu',
        width=int(WINDOW_SIZE[0] * 0.6)
    )

    main_menu.add.button('Play', play_menu)
    main_menu.add.button('Quit', pygame_menu.events.EXIT)

    return main_menu

def handle_events() -> None:
    events = pygame.event.get()
    for event in events:
        if event.type == pygame.QUIT:
            exit()

if __name__ == '__main__':
    main_program()



Selected gamemode: "('2 - Connect5', 'CONNECT5')" (CONNECT5) at index 1


TypeError: 'str' object does not support item assignment